### 2.6 比较运算符方法  
当设计比较运算符时，需要考虑两个因素  
- 1. 如何比较同一个类的两个对象  
- 2. 如何比较不同类的对象  

> 简洁的并不一定是最好的 

In [8]:
"""
同一个类的对象比较
比较运算符至少要实现一组 
"""

class BlackJackCard:
    def __init__(self, rank, suit, hard, soft):
        """
        init a blackcard object 
        :param: rank  牌号 
        :param: suit  花色
        :hard
        :soft
        """
        self.rank = rank 
        self.suit = suit 
        self.hard = hard 
        self.soft = soft 

    def __lt__(self, other):
        """
        less than : < 
        """
        # 显式的类型检查
        if not isinstance(other, BlackJackCard): 
            return NotImplemented 
        return self.rank < other.rank  
    
    def __le__(self, other):
        """
        less or equal than : <= 
        """
        # 隐式类型检查
        try:
            return self.rank <= other.rank 
        except AttributeError:
            return NotImplemented 

    def __eq__(self, other):
        """
        equal: = 
        """
        if not isinstance(other, BlackJackCard):
            return NotImplemented 
        return self.rank == other.rank and self.suit == other.suit 

    def __ne__(self, other):
        """
        not equal: ≠
        """
        if not isinstance(other, BlackJackCard):
            return NotImplemented 
        return self.rank != other.rank or self.suit != other.rank 

    def __str__(self):
        return "{__class__.__name__}, rank={rank!r}, suit={suit!r}".format(__class__=self.__class__, **self.__dict__)

理论上，try语句块有一个优点： 避免了重复的类命名 

In [9]:
from functools import partial 

class NumberCard(BlackJackCard):
    def __init__(self, rank, suit):
        super().__init__(rank, suit, int(rank), int(rank))

class FaceCard(BlackJackCard):
    def __init__(self, rank, suit):
        super().__init__(rank, suit, 10, 10)

class AceCard(BlackJackCard):
    def __init__(self, rank, suit):
        super().__init__(rank, suit, 1, 11)

def card21(rank, suit):
    """ 工厂方法
    """
    class_ = {
        1: partial(AceCard, rank),
        'J': partial(FaceCard, 11),
        'Q': partial(FaceCard, 12),
        'K': partial(FaceCard, 13)
    }.get(rank, partial(NumberCard, rank))

    return class_(suit)

two = card21(2, '♠')
three = card21(3, '♥')
two_c = card21(2, '♣')
eleven = card21('J', '♠')


In [10]:
two == two_c
#two < 2 # 会抛出有异常 
two == 2 # NotImplemented 抛出异常后会尝试交换顺序，由于默认有int.__eq__()方法，因此可以正常的执行 

False

通常情况下， 一个好的子类定义应该只以来于相同的方法签名   
而不合理的多态设计则会广泛的使用 isinstance   
在一些情况下， 尤其是在使用Python的内置类时，使用isinstance 时必须 的

In [21]:
"""
不同类型之间比较的一个例子
"""
class Hand:
    def __init__(self, dealer_card, *cards):
        self.dealer_card = dealer_card 
        self.cards = list(cards) 

    def __str__(self):
        """ 返回所有手牌"""        
        return ','.join(map(str, self.cards))

    def __repr__(self):
        """ 返回类名及参数值 """
        return "{__class__.__name__} dealer_card={deal_card!r}, _card_str={_card_str}".format(
            __clas__ = self.__class__, 
            _card_str = ','.join(map(str, repr(self.cards))),
            **self.__dict__
        )
    
    def __eq__(self, other):
        """
         = 重载
         other: an instance of Int or Hand 
        """
        if isinstance(other, int):
            return self.total() == other 
        try:
            return self.cards == other.cards and self.dealer_card == other.dealer_card 
        except AttributeError:
            return NotImplemented  # 表示没有实现该组类型对象的比较  

    def __lt__(self, other):
        """
        < 重载 
        """ 
        if isinstance(other, int):
            return self.total() < other 
        try:
            return self.total() < other.total()
        except AttributeError:
            raise NotImplemented
    
    def __le__(self, other):
        """
        <= 重载 
        """
        if isinstance(other, int):
            return self.total() <= other 
        try:
            return self.total() <= other.total()
        except AttributeError:
            raise NotImplemented
    
    def total(self):
        """ 计算手牌的soft总和 和 hard总和, 可以被外部调用
        """
        delta_soft = max(card.soft - card.rank for card in self.cards)
        hard = sum(card.rank for card in self.cards)
        if hard + delta_soft <= 21:
            return hard + delta_soft 
        else:
            return hard 


In [22]:
two = card21(2, '♠')
three = card21(3, '♠')
two_c = card21(2, '♣')
ace = card21(1, '♠') 
cards = [ace, two, two_c, three] # 这作者也是严谨，按牌的大小组合手牌 

h = Hand(card21(10, '♠'), *cards)
print(h)
print(h.total())

AceCard, rank=1, suit='♠',NumberCard, rank=2, suit='♠',NumberCard, rank=2, suit='♣',NumberCard, rank=3, suit='♠'
18


In [23]:
h2 = Hand(card21(10, '♠'), card21(5, '♠'), *cards)
h2.total()

13

In [24]:
h < h2 

False

### 2.7 析构函数__del__方法

该方法的目的是在将一个对象从内存中清除前，可以有机会做一些操作    
如果一个python 对象包含了一些操作系统的资源,\_\_del\_\_()方法是将其释放的最后机会   

> 由于析构函数的不稳定性， 通常更建议使用上下文管理器来进行对象的释放 

Cpython的实现中，对象会包括一个引用计数器， 当对象被赋值给变量时， 计数器加一，变量删除时，计数器减一。当引用计数器为0时，表示改对象可以销毁 

In [14]:
class Noisy:
    def __del__(self):
        print("Removing {0}".format(id(self))) 

x = Noisy() 
del x 

Removing 140009616993392


In [15]:
ln = [Noisy(), Noisy()]
ln2 = ln[:]
del ln

In [16]:
del ln2

Removing 140009693820096
Removing 140009693820768
